# 背景
进行一项某项药物临床实验时，实验关注的是药物的正面效果，但是药物也可能会有严重的副作用，如何进行监控呢？      
衡量药物是否有效通过假设检验来判断，同理容易想到也可以通过假设检验判断药物的负面作用。但是固定水平检验偷看会增加假阳性，如果提前分析会增加假阳性；如果按照正确流程在实验结束后分析，实验中的志愿者可能已经受到了严重的伤害。      
如何完成分析又不增加假阳性呢？医学上的做法叫[期中分析](https://baike.baidu.com/item/%E6%9C%9F%E4%B8%AD%E5%88%86%E6%9E%90/5775022?fr=aladdin)，依赖于成组贯序分析，[alpha消耗函数](https://www.analytics-toolkit.com/glossary/alpha-spending/)是其中一种。      

# 解决思路
如果只是为了控制假阳性，可以将提前比较视为多次比较，使用[多重检验](https://www.jianshu.com/p/2d30efc37b8b)方法进行校正。但是实验中数据相关性较强，多重校正会增大假阴性错误，并不合适。      
alpha消耗函数的思路是将假阳性错误按照某种方案分配给每次比较，**每次比较消耗一定的假阳性配额，合计后刚好等于预设水平**。      

# 消耗函数
设实验对应时间为$[0, T]$，在这期间我们会得到对应信息$[0, I]$。      
观测时间为：$0, t_1, t_2, t_3...T$；      
对应信息为：$0, i_1, i_2, i_3...I$；      
信息量占比为($t^*$)：$0, t^*_1 = i_1/I, t^*_2 = i_2/I, t^*_3 = i_3/I, ... 1$；      
alpha消耗函数为$t^*$的函数需要满足:      
$$\alpha(0) = 0$$
$$\alpha(1) = \alpha$$      
每次消耗α为：$α^*_1 = \alpha(t^*_1) - \alpha(0), α^*_2 = \alpha(t^*_2) - \alpha(t^*_1),...,α^*_n = \alpha(1) - \alpha(t^*_{n - 1})$。    
    
根据定义可知$α^*_1 = \alpha(t^*_1)$，且$\sum \alpha^* = \alpha$，符合解决思路中的要求。    

# 如何联系背景问题

假阳性是在实际无效果情况下发现显著。以下仅以单样本单尾为例，根据不等式求解每个时刻的显著边界$Z_c$:   

$$\\t_1:P\{Z(t^*_1) >= Z_c(t^*_1)\}= \alpha(t^*_1)$$    

$$t_2:P\{Z(t^*_2) >= Z_c(t^*_2) , Z(t^*_1) < Z_c(t^*_1)\} \\= P\{Z_c(t^*_2) | Z(t^*_1) )< Z_c(t^*_1)\} * (1 -  \alpha(t^*_1))\\= \alpha(t^*_2) -  \alpha(t^*_1)$$    

...

$$t_n:P\{Z(t^*_n) >= Z_c(t^*_n) , Z(t^*_1) < Z_c(t^*_1),Z(t^*_2) < Z_c(t^*_2)...,Z(t^*_{n -1}) < Z_c(t^*_{n -1})\}  \\= P\{Z(t^*_n) >= Z_c(t^*_n) | Z(t^*_1) < Z_c(t^*_1),Z(t^*_2) < Z_c(t^*_2)...,Z(t^*_{n-1}) < Z_c(t^*_{n-1})\}  * (1 - \alpha(t^*_{n -1}))\\= \alpha -  \alpha(t^*_{n-1})$$

若检测时大于显著边界${Z_c}$则认为显著，接受备择假设。

# 如何求解${Z_c}$
在样本中个体独立同分布的情况下，信息量$I$为预设总样本量，每次检验的信息量$i$为已完成检验的个体数量。    
求解边界并不容易，仅有第一次检测边界可以直接求出：    

$$t_1:Z_c(t^*_1) = \phi^{-1}(\alpha(t^*_1))$$
    
后续边界该如何求解？    
    
随着实验进行，若每次检验新增抽样都比较大，则新抽样样本的均值与和都服从正态分布，累计值的变化过程服从布朗运动，可以通过伊藤积分或蒙特卡洛模拟进行求解，以下为蒙特卡洛求解的过程：    

1.随机产生N个样本，每个样本有M个标准正态分布抽样的个体，M与N都极大；
2.可直接求得边界$Z_c(t^*_1)$，对每个样本的前 $M * t^*_1$个体合计计算$Z$值，不符合条件的样本标记淘汰；
3.计算第二次的alpha控制目标$\alpha(t^*_2) -  \alpha(t^*_1)$，对未淘汰的样本的前 $M * t^*_2$个体合计计算$Z$值，排序后取对应分位点的$Z$值近似认为$Z_c(t^*_2)$，同样将小于边界的样本标记淘汰；
4.重复上述过程，直至完成。
    
通过上述过程，得到了显著边界序列$[Z_c(t^*_1), Z_c(t^*_2), Z_c(t^*_3)... Z_c(t^*_n)]$，可以用作每次检测的边界。
    
# 后记
中文网站上关于成组贯序分析的介绍比较少，尤其时alpha消耗函数法更少，以上是阅读相关论文后的个人理解。    
本文仅介绍了最简单的情况，且没有详细介绍为何服从布朗运动，详细推导和展开请继续查阅相关英文资料。    